In [1]:
!pip install pyspark

You should consider upgrading via the '/home/fmenegolo/.asdf/installs/python/3.9.1/bin/python3.9 -m pip install --upgrade pip' command.
Reshimming asdf python...


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import col, substring, when, regexp_replace, size, expr
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType


spark = SparkSession \
    .builder \
    .master("local[*]")\
    .appName("ELT TRUSTED") \
    .getOrCreate()


/usr/sbin/hadoop


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/apache-spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/27 00:59:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/27 00:59:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Definir o esquema para o DataFrame df_movies_IMDB
schema_movies_IMDB = StructType([
    StructField("id", StringType(), True),
    StructField("tituloPincipal", StringType(), True),
    StructField("tituloOriginal", StringType(), True),
    StructField("anoLancamento", StringType(), True),
    StructField("tempoMinutos", StringType(), True),
    StructField("genero", StringType(), True),
    StructField("notaMedia", FloatType(), True),  # Definir tipo como FloatType()
    StructField("numeroVotos", IntegerType(), True),  # Definir tipo como IntegerType()
    StructField("generoArtista", StringType(), True),
    StructField("personagem", StringType(), True),
    StructField("nomeArtista", StringType(), True),
    StructField("anoNascimento", StringType(), True),  
    StructField("anoFalecimento", StringType(), True),  
    StructField("profissao", StringType(), True),
    StructField("titulosMaisConhecidos", StringType(), True)
])

# Definir o esquema para o DataFrame df_series_IMDB
schema_series_IMDB = StructType([
    StructField("id", StringType(), True),
    StructField("tituloPincipal", StringType(), True),
    StructField("tituloOriginal", StringType(), True),
    StructField("anoLancamento", StringType(), True), 
    StructField("anoTermino", StringType(), True),  
    StructField("tempoMinutos", IntegerType(), True),  # Definir tipo como IntegerType()
    StructField("genero", StringType(), True),
    StructField("notaMedia", FloatType(), True),  # Definir tipo como FloatType()
    StructField("numeroVotos", IntegerType(), True),  # Definir tipo como IntegerType()
    StructField("generoArtista", StringType(), True),
    StructField("personagem", StringType(), True),
    StructField("nomeArtista", StringType(), True),
    StructField("anoNascimento", StringType(), True),  
    StructField("anoFalecimento", StringType(), True),  
    StructField("profissao", StringType(), True),
    StructField("titulosMaisConhecidos", StringType(), True)
])

In [4]:
path1 = f"/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_7/Desafio01_ETL/assets/"
df_movies_IMDB = spark.read.option("delimiter", "|").schema(schema_movies_IMDB).csv(path1 + 'movies.csv', header=True)
df_series_IMDB = spark.read.option("delimiter", "|").schema(schema_series_IMDB).csv(path1 + 'series.csv', header=True)

path2 = f"/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_8/Desafio02_ETL/Tarefa_02/assets/"
df_movies_TMDB = spark.read.json(path2 + 'movies.json') 
df_series_TMDB = spark.read.json(path2 + 'series.json')

In [5]:
df_movies_IMDB = df_movies_IMDB.withColumnRenamed("tituloPincipal", "tituloPrincipal")
df_series_IMDB = df_series_IMDB.withColumnRenamed("tituloPincipal", "tituloPrincipal")

## 2. 
 Filtro os Filmes do BD Local com lista de filmes do TMDB

In [6]:

# Converter a coluna 'title' do DataFrame JSON para um array de palavras e extrair o ano de lançamento
df_movies_TMDB_titulos_datas = df_movies_TMDB.select(
    regexp_replace(col('title'), r"[^a-zA-Z0-9\s]", "").alias('title_replaced'),
    substring(col('release_date'), 1, 4).alias('releaseDate')
)

# Aplicar substituição nos títulos
df_movies_IMDB = df_movies_IMDB.withColumn('tituloPrincipal_replaced', regexp_replace(col('tituloPrincipal'), r"[^a-zA-Z0-9\s]", ""))

# Realizar a filtragem dos títulos
df_movies_IMDB_trusted = df_movies_IMDB.join(df_movies_TMDB_titulos_datas,
    expr(
        "(size(split(tituloPrincipal_replaced, ' ')) = 1 AND " +
        "size(filter(split(tituloPrincipal_replaced, ' '), x -> length(x) > 4 AND instr(title_replaced, x) > 0)) >= 1) OR " +
        "(size(split(tituloPrincipal_replaced, ' ')) = 2 AND " +
        "size(filter(split(tituloPrincipal_replaced, ' '), x -> length(x) >= 1 AND instr(title_replaced, x) > 0)) >= 2) OR " +
        "(size(split(tituloPrincipal_replaced, ' ')) >= 3 AND " +
        "size(filter(split(tituloPrincipal_replaced, ' '), x -> length(x) > 3 AND instr(title_replaced, x) > 0)) >= 2)"
    ) &
    (df_movies_IMDB['anoLancamento'] == df_movies_TMDB_titulos_datas['releaseDate']) &
    (col('tituloPrincipal_replaced').isNotNull()) &
    (col('tituloPrincipal_replaced') != '') &
    (col('title_replaced').isNotNull()) &
    (col('title_replaced') != ''),
    'inner')

# Remover as colunas 'title' e 'releaseDate' do DataFrame
#df_movies_IMDB_trusted = df_movies_IMDB_trusted.drop('title_replaced', 'releaseDate')

# Trata os valores "NA" na coluna "anoFalecimento"
df_movies_IMDB_trusted = df_movies_IMDB_trusted.withColumn("anoFalecimento", when(col("anoFalecimento") == "\\N", None).otherwise(col("anoFalecimento")))
df_movies_IMDB_trusted = df_movies_IMDB_trusted.withColumn("anoNascimento", when(col("anoNascimento") == "\\N", None).otherwise(col("anoNascimento")))
df_movies_IMDB_trusted = df_movies_IMDB_trusted.withColumn("tempoMinutos", when(col("tempoMinutos") == "\\N", None).otherwise(col("tempoMinutos")))
df_movies_IMDB_trusted = df_movies_IMDB_trusted.withColumn("notaMedia", when(col("notaMedia") == "\\N", None).otherwise(col("notaMedia")))
df_movies_IMDB_trusted = df_movies_IMDB_trusted.withColumn("numeroVotos", when(col("numeroVotos") == "\\N", None).otherwise(col("numeroVotos")))

# Elimina possiveis Redundâncias
df_movies_IMDB_trusted = df_movies_IMDB_trusted.distinct()

In [7]:

df_movies_IMDB_trusted.show(10)
print("Número de linhas antes Filmes IMDB: ", df_movies_IMDB.count())
print("Número de linhas no resultado: ", df_movies_IMDB_trusted.count())
count = df_movies_IMDB_trusted.select('title_replaced').distinct().count()
print("Número de Filmes Relacionados no resultado: ", count)

+----------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+-------------+--------------------+------------------+-------------+--------------+--------------------+---------------------+------------------------+--------------------+-----------+
|        id|     tituloPrincipal|      tituloOriginal|anoLancamento|tempoMinutos|              genero|notaMedia|numeroVotos|generoArtista|          personagem|       nomeArtista|anoNascimento|anoFalecimento|           profissao|titulosMaisConhecidos|tituloPrincipal_replaced|      title_replaced|releaseDate|
+----------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+-------------+--------------------+------------------+-------------+--------------+--------------------+---------------------+------------------------+--------------------+-----------+
| tt0405821|            Casshern|            Casshern|         2004|     

Número de linhas antes Filmes IMDB:  1045161


Número de linhas no resultado:  166


Número de Filmes Relacionados no resultado:  28


## 3. 
Filtro as Series do BD Local com lista de series do TMDB

In [8]:
# Converter a coluna 'name' e a substring de 'first_air_date' do DataFrame JSON para o DataFrame do TMDB
df_series_TMDB_titulos_datas = df_series_TMDB.select(regexp_replace(col('name'), r"[^a-zA-Z0-9\s]", "").alias('title_replaced'),
    substring(col('first_air_date'), 1, 4).alias('releaseDate'))

# Aplicar substituição nos títulos
df_series_IMDB = df_series_IMDB.withColumn('tituloPrincipal_replaced', regexp_replace(col('tituloPrincipal'), r"[^a-zA-Z0-9\s]", ""))

# Realizar a filtragem dos títulos e anos de lançamento

df_series_IMDB_layer1 = df_series_IMDB.join(df_series_TMDB_titulos_datas,
    expr(
        "(size(split(tituloPrincipal_replaced, ' ')) = 1 AND " +
        "size(filter(split(tituloPrincipal_replaced, ' '), x -> length(x) > 3 AND instr(title_replaced, x) > 0)) >= 1) OR " +
        "(size(split(tituloPrincipal_replaced, ' ')) = 2 AND " +
        "size(filter(split(tituloPrincipal_replaced, ' '), x -> length(x) >= 1 AND instr(title_replaced, x) > 0)) >= 2) OR " +
        "(size(split(tituloPrincipal_replaced, ' ')) >= 3 AND " +
        "size(filter(split(tituloPrincipal_replaced, ' '), x -> length(x) > 3 AND instr(title_replaced, x) > 0)) >= 2)"
    ) &
    (df_series_IMDB['anoLancamento'] == df_series_TMDB_titulos_datas['releaseDate']) &
    (col('tituloPrincipal_replaced').isNotNull()) &
    (col('tituloPrincipal_replaced') != '') &
    (col('title_replaced').isNotNull()) &
    (col('title_replaced') != ''),
    'inner')

# Remover as colunas 'title' e 'releaseDate' do DataFrame
#df_series_IMDB_layer1 = df_series_IMDB_layer1.drop('title_replaced', 'releaseDate')

df_series_IMDB_layer1 = df_series_IMDB_layer1.withColumn("anoTermino", when(col("anoTermino") == "\\N", None).otherwise(col("anoTermino")))
df_series_IMDB_layer1 = df_series_IMDB_layer1.withColumn("tempoMinutos", when(col("tempoMinutos") == "\\N", None).otherwise(col("tempoMinutos")))
df_series_IMDB_layer1 = df_series_IMDB_layer1.withColumn("notaMedia", when(col("notaMedia") == "\\N", None).otherwise(col("notaMedia")))
df_series_IMDB_layer1 = df_series_IMDB_layer1.withColumn("numeroVotos", when(col("numeroVotos") == "\\N", None).otherwise(col("numeroVotos")))
df_series_IMDB_layer1 = df_series_IMDB_layer1.withColumn("anoNascimento", when(col("anoNascimento") == "\\N", None).otherwise(col("anoNascimento")))
df_series_IMDB_layer1 = df_series_IMDB_layer1.withColumn("anoFalecimento", when(col("anoFalecimento") == "\\N", None).otherwise(col("anoFalecimento")))
df_series_IMDB_layer1 = df_series_IMDB_layer1.withColumn("anoFalecimento", when(col("anoFalecimento") == "\\N", None).otherwise(col("anoFalecimento")))

# Elimina possiveis Redundâncias
df_series_IMDB_layer1 = df_series_IMDB_layer1.distinct()



In [9]:

df_series_IMDB_layer1.show(10)
print("Número de linhas antes Series IMDB: ", df_series_IMDB.count())
print("Número de linhas no resultado: ", df_series_IMDB_layer1.count())
count = df_series_IMDB_layer1.select('title_replaced').distinct().count()
print("Número de Series Relacionadas no resultado: ", count)

+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+-----------------+-------------+--------------+------------------+---------------------+------------------------+--------------------+-----------+
|       id|     tituloPrincipal|      tituloOriginal|anoLancamento|anoTermino|tempoMinutos|              genero|notaMedia|numeroVotos|generoArtista|          personagem|      nomeArtista|anoNascimento|anoFalecimento|         profissao|titulosMaisConhecidos|tituloPrincipal_replaced|      title_replaced|releaseDate|
+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+-----------------+-------------+--------------+------------------+---------------------+------------------------+--------------------+-----------+
|tt0078570|The Rose of Versa...|   Berusaiyu no bara

Número de linhas no resultado:  32023


Número de Series Relacionadas no resultado:  3136


## 4. 
Filtro as Series do TMDB com lista de Filmes do TMDB


In [10]:

# Aplicar substituição nos títulos
df_series_TMDB = df_series_TMDB.withColumn('name_replaced', regexp_replace(col('name'), r"[^a-zA-Z0-9\s]", ""))

# Realizar a filtragem dos títulos
df_series_TMDB_trusted = df_series_TMDB.join(df_movies_TMDB_titulos_datas,
    expr(
        "(size(split(name_replaced, ' ')) = 1 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) > 4 AND instr(title_replaced, x) > 0)) >= 1) OR " +
        "(size(split(name_replaced, ' ')) = 2 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) >= 1 AND instr(title_replaced, x) > 0)) >= 2) OR " +
        "(size(split(name_replaced, ' ')) >= 3 AND " +
        "size(filter(split(name_replaced, ' '), x -> length(x) > 3 AND instr(title_replaced, x) > 0)) >= 2)"
    ) &
    (col('name_replaced').isNotNull()) &
    (col('name_replaced') != '') &
    (col('title_replaced').isNotNull()) &
    (col('title_replaced') != ''),
    'inner')

# Retira colunas irrelevantes
df_series_TMDB_trusted = df_series_TMDB_trusted.drop('titulo','backdrop_path','poster_path')
#df_series_TMDB_trusted = df_series_TMDB_trusted.drop('title_replaced','releaseDate')

# Trata os valores "NA" nas colunas
df_series_TMDB_trusted = df_series_TMDB_trusted.withColumn("first_air_date", when(col("first_air_date") == "", None).otherwise(col("first_air_date")))
df_series_TMDB_trusted = df_series_TMDB_trusted.withColumn("genre_ids", when(size(col("genre_ids")) == 0, None).otherwise(col("genre_ids")))
df_series_TMDB_trusted = df_series_TMDB_trusted.withColumn("overview", when(col("overview") == "", None).otherwise(col("overview")))
df_series_TMDB_trusted = df_series_TMDB_trusted.withColumn("popularity", when(col("popularity") == "", None).otherwise(col("popularity")))
df_series_TMDB_trusted = df_series_TMDB_trusted.withColumn("vote_average", when(col("vote_average") == "", None).otherwise(col("vote_average")))
df_series_TMDB_trusted = df_series_TMDB_trusted.withColumn("vote_count", when(col("vote_count") == "", None).otherwise(col("vote_count")))

# Elimina possiveis Redundâncias
df_series_TMDB_trusted = df_series_TMDB_trusted.distinct()

In [11]:
df_series_TMDB_trusted.show(10)
print("Número de linhas antes Series TMDB: ", df_series_TMDB.count())
print("Número de linhas no resultado: ", df_series_TMDB_trusted.count())
count = df_series_TMDB_trusted.select('title_replaced').distinct().count()
print("Número de Filmes relacionados no resultado2: ", count)

+--------------+--------------------+------+--------------------+--------------+-----------------+----------------------------------+--------------------+----------+------------+----------+--------------------+--------------------+-----------+
|first_air_date|           genre_ids|    id|                name|origin_country|original_language|                     original_name|            overview|popularity|vote_average|vote_count|       name_replaced|      title_replaced|releaseDate|
+--------------+--------------------+------+--------------------+--------------+-----------------+----------------------------------+--------------------+----------+------------+----------+--------------------+--------------------+-----------+
|    2009-06-24|         [16, 10759]| 61389|Saint Seiya: The ...|          [JP]|               ja|         聖闘士星矢 THE LOST CA...|An orphan known a...|    47.146|         8.2|       529|Saint Seiya The L...|Os Cavaleiros do ...|       2023|
|    2018-01-16|          [18

Número de linhas no resultado:  62


Número de Filmes relacionados no resultado2:  25


## 5. 
Filtro as Series do BD Local layer1 que foram filtradas no passo 3

com a dos Series do TMDB Trusted que foram filtradas no passo 4

In [12]:
# Converter a coluna 'title' do DataFrame JSON para um array de palavras e extrair o ano de lançamento
df_series_TMDB_titulos_datas = df_series_TMDB_trusted.select(regexp_replace(col('name'), r"[^a-zA-Z0-9\s]", "").alias('title_replaced'),
    substring(col('first_air_date'), 1, 4).alias('releaseDate'))

# Drop na coluna para evitar ambiguidade (somente para versao local)
df_series_IMDB_layer1 = df_series_IMDB_layer1.drop('title_replaced', 'releaseDate')

# Realizar a filtragem dos títulos e anos de lançamento
df_series_IMDB_trusted = df_series_IMDB_layer1.join(df_series_TMDB_titulos_datas,
    expr(
        "(size(split(tituloPrincipal_replaced, ' ')) = 1 AND " +
        "size(filter(split(tituloPrincipal_replaced, ' '), x -> length(x) > 3 AND instr(title_replaced, x) > 0)) >= 1) OR " +
        "(size(split(tituloPrincipal_replaced, ' ')) = 2 AND " +
        "size(filter(split(tituloPrincipal_replaced, ' '), x -> length(x) >= 1 AND instr(title_replaced, x) > 0)) >= 2) OR " +
        "(size(split(tituloPrincipal_replaced, ' ')) >= 3 AND " +
        "size(filter(split(tituloPrincipal_replaced, ' '), x -> length(x) > 3 AND instr(title_replaced, x) > 0)) >= 2)"
    ) &
    (df_series_IMDB['anoLancamento'] == df_series_TMDB_titulos_datas['releaseDate']) &
    (col('tituloPrincipal_replaced').isNotNull()) &
    (col('tituloPrincipal_replaced') != '') &
    (col('title_replaced').isNotNull()) &
    (col('title_replaced') != ''),
    'inner')

# Retira colunas irrelevantes (somete para versão AWS)
#df_series_IMDB_layer1 = df_series_IMDB_layer1.drop('title_replaced', 'releaseDate')

# Trata os valores "NA" nas colunas
df_series_IMDB_trusted = df_series_IMDB_trusted.withColumn("anoTermino", when(col("anoTermino") == "\\N", None).otherwise(col("anoTermino")))
df_series_IMDB_trusted = df_series_IMDB_trusted.withColumn("tempoMinutos", when(col("tempoMinutos") == "\\N", None).otherwise(col("tempoMinutos")))
df_series_IMDB_trusted = df_series_IMDB_trusted.withColumn("genero", when(col("genero") == "\\N", None).otherwise(col("genero")))
df_series_IMDB_trusted = df_series_IMDB_trusted.withColumn("notaMedia", when(col("notaMedia") == "\\N", None).otherwise(col("notaMedia")))
df_series_IMDB_trusted = df_series_IMDB_trusted.withColumn("numeroVotos", when(col("numeroVotos") == "\\N", None).otherwise(col("numeroVotos")))
df_series_IMDB_trusted = df_series_IMDB_trusted.withColumn("anoNascimento", when(col("anoNascimento") == "\\N", None).otherwise(col("anoNascimento")))
df_series_IMDB_trusted = df_series_IMDB_trusted.withColumn("anoNascimento", when(col("anoNascimento") == "\\N", None).otherwise(col("anoNascimento")))
df_series_IMDB_trusted = df_series_IMDB_trusted.withColumn("anoFalecimento", when(col("anoFalecimento") == "\\N", None).otherwise(col("anoFalecimento")))


# Remover linhas duplicadas no resultado
df_series_IMDB_trusted = df_series_IMDB_trusted.distinct()


In [13]:
df_series_IMDB_trusted.show(10)
print("Número de linhas antes IMDB Layer 1: ", df_series_IMDB_layer1.count())
print("Número de linhas no resultado: ", df_series_IMDB_trusted.count())
count = df_series_IMDB_trusted.select('title_replaced').distinct().count()
print("Número de Series Relacionas no resultado: ", count)

## 6.
Elimina algumas colunas não relevantes do df_movies_TMDB 

In [ ]:
# remove os colunas não relevantes


df_movies_TMDB_trusted = df_movies_TMDB.drop('adult','backdrop_path','poster_path','video')
df_movies_TMDB_trusted = df_movies_TMDB_trusted.withColumn("popularity", when(col("popularity") == "", None).otherwise(col("popularity")))
df_movies_TMDB_trusted = df_movies_TMDB_trusted.withColumn("vote_average", when(col("vote_average") == "", None).otherwise(col("vote_average")))
df_movies_TMDB_trusted = df_movies_TMDB_trusted.withColumn("vote_count", when(col("vote_count") == "", None).otherwise(col("vote_count")))
df_movies_TMDB_trusted = df_movies_TMDB_trusted.withColumn("popularity", col("popularity").cast(FloatType()))
df_movies_TMDB_trusted = df_movies_TMDB_trusted.withColumn("vote_average", col("vote_average").cast(FloatType()))
df_movies_TMDB_trusted = df_movies_TMDB_trusted.withColumn("vote_count", col("vote_count").cast(IntegerType()))
df_series_TMDB_trusted = df_series_TMDB_trusted.withColumn("popularity", col("popularity").cast(FloatType()))
df_series_TMDB_trusted = df_series_TMDB_trusted.withColumn("vote_average", col("vote_average").cast(FloatType()))
df_series_TMDB_trusted = df_series_TMDB_trusted.withColumn("vote_count", col("vote_count").cast(IntegerType()))

df_movies_TMDB.show(5)
df_movies_TMDB_trusted
df_movies_TMDB_trusted.show(5)
df_movies_TMDB_trusted = df_series_IMDB_trusted.distinct()
#print("Número de linhas antes tMDB: ", df_movies_TMDB_trusted.count())

+-----+--------------------+-------------+------+-----------------+--------------------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|adult|       backdrop_path|    genre_ids|    id|original_language|                  original_title|            overview|popularity|         poster_path|release_date|               title|video|vote_average|vote_count|
+-----+--------------------+-------------+------+-----------------+--------------------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|false|/kgATFkG4SDyengNM...| [14, 28, 12]|455476|               en|            Knights of the Zo...|Quando um órfão d...|   292.303|/omPLRVEydDJdkXoB...|  2023-04-27|Os Cavaleiros do ...|false|         6.3|        49|
|false|/5hoS3nEkGGXUfmnu...| [28, 12, 14]|960704|               ja|  鋼の錬金術師 完結編 最後の錬成|Acompanhe a últim...|    82.899|/uMzOYHGiH

## 7.
escreve os dados na camada Trusted no S3 no formato parquet


In [ ]:
"""

# Obtém a data atual para criar os diretórios correspondentes
current_date = datetime.now()
ano = current_date.strftime("%Y")
mes = current_date.strftime("%m")
dia = current_date.strftime("%d")

# Define os caminhos de destino no S3 para os DataFrames no formato Parquet
path_trt_local_movies = "s3://data-lake-do-fabricio/TRT/Local/Filmes/{ano}/{mes}/{dia}/"
path_trt_local_series = "s3://data-lake-do-fabricio/TRT/Local/Series/{ano}/{mes}/{dia}/"
path_trt_tmdb_movies = "s3://data-lake-do-fabricio/TRT/TMDB/Filmes/{ano}/{mes}/{dia}/"
path_trt_tmdb_series = "s3://data-lake-do-fabricio/TRT/TMDB/Series/{ano}/{mes}/{dia}/"

# Salva os DataFrames no formato Parquet no S3
df_movies_IMDB_trusted.write.parquet(path_trt_local_movies.format(ano=ano, mes=mes, dia=dia))
df_series_IMDB_trusted.write.parquet(path_trt_local_series.format(ano=ano, mes=mes, dia=dia))
df_movies_TMDB_trusted.write.parquet(path_trt_tmdb_movies.format(ano=ano, mes=mes, dia=dia))
df_series_TMDB_trusted.write.parquet(path_trt_tmdb_series.format(ano=ano, mes=mes, dia=dia))

"""

'\n\n# Obtém a data atual para criar os diretórios correspondentes\ncurrent_date = datetime.now()\nano = current_date.strftime("%Y")\nmes = current_date.strftime("%m")\ndia = current_date.strftime("%d")\n\n# Define os caminhos de destino no S3 para os DataFrames no formato Parquet\npath_trt_local_movies = "s3://data-lake-do-fabricio/TRT/Local/Filmes/{ano}/{mes}/{dia}/"\npath_trt_local_series = "s3://data-lake-do-fabricio/TRT/Local/Series/{ano}/{mes}/{dia}/"\npath_trt_tmdb_movies = "s3://data-lake-do-fabricio/TRT/TMDB/Filmes/{ano}/{mes}/{dia}/"\npath_trt_tmdb_series = "s3://data-lake-do-fabricio/TRT/TMDB/Series/{ano}/{mes}/{dia}/"\n\n# Salva os DataFrames no formato Parquet no S3\ndf_movies_IMDB_trusted.write.parquet(path_trt_local_movies.format(ano=ano, mes=mes, dia=dia))\ndf_series_IMDB_trusted.write.parquet(path_trt_local_series.format(ano=ano, mes=mes, dia=dia))\ndf_movies_TMDB_trusted.write.parquet(path_trt_tmdb_movies.format(ano=ano, mes=mes, dia=dia))\ndf_series_TMDB_trusted.writ